In [4]:
from osgeo import gdal

# File without an extension
data_file = r"D:\wenqu\2024_data\site6\site6_Hyper\100221_site6_flight3_2024_07_18_19_21_56\raw_34000_rd"

# Open the dataset
dataset = gdal.Open(data_file)

# Get projection information
projection = dataset.GetProjection()

# Print results
if projection:
    print("Projection system (WKT):")
    print(projection)
else:
    print("No projection system found in the binary file.")

# Get geotransform information (spatial resolution, origin)
geo_transform = dataset.GetGeoTransform()
if geo_transform:
    print("\nGeoTransform:")
    print(geo_transform)
else:
    print("No GeoTransform information found.")

# Close the dataset
dataset = None


No projection system found in the binary file.

GeoTransform:
(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [2]:
import rasterio
import numpy as np

# Open AVIRIS-NG hyperspectral image
with rasterio.open(r"D:\wenqu\chapter1_2\updated_aviris\site2a\ang20190707t023526_rfl_v2v2\ang20190707t023526_rfl_v2v2_img") as src:
    meta = src.meta
    wavelengths = src.tags()['Wavelength'].strip('[]').split(',')  # Optional, depends on image tags
    wavelengths = np.array(wavelengths, dtype=float)  # Convert to numeric if available

    # You can use band index if wavelengths are not available
red_idx = np.argmin(np.abs(wavelengths - 670)) + 1  # rasterio bands are 1-indexed
nir_idx = np.argmin(np.abs(wavelengths - 850)) + 1
red = src.read(red_idx).astype(float)
nir = src.read(nir_idx).astype(float)
ndvi = (nir - red) / (nir + red)

KeyError: 'Wavelength'

In [ ]:
meta.update(dtype=rasterio.float32, count=1)

with rasterio.open("D:\wenqu\chapter1_2\updated_aviris\NDVI\site2a_ndvi_output.tif", "w", **meta) as dst:
    dst.write(ndvi.astype(rasterio.float32), 1)


In [1]:
from osgeo import gdal
import numpy as np

# Open AVIRIS-NG image
img_path = r"D:\wenqu\chapter1_2\updated_aviris\site3b\ang20220709t184317rfl\ang20220709t184317_rfl_v2aa2_img"
ds = gdal.Open(img_path)

# RED ≈ band 59, NIR ≈ band 95
red_band = ds.GetRasterBand(59).ReadAsArray().astype(float)
nir_band = ds.GetRasterBand(95).ReadAsArray().astype(float)

# NDVI calculation
ndvi = (nir_band - red_band) / (nir_band + red_band)
ndvi = np.clip(ndvi, -1, 1)

# Save NDVI
driver = gdal.GetDriverByName('GTiff')
out_path = r"D:\wenqu\chapter1_2\updated_aviris\NDVI\site3b_ndvi_output.tif"
out_ds = driver.Create(out_path, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)

# Set geotransform & projection
out_ds.SetGeoTransform(ds.GetGeoTransform())
out_ds.SetProjection(ds.GetProjection())

# Write NDVI band
out_ds.GetRasterBand(1).WriteArray(ndvi)
out_ds.GetRasterBand(1).SetNoDataValue(-9999)
out_ds.FlushCache()

print("NDVI saved to:", out_path)


NDVI saved to: D:\wenqu\chapter1_2\updated_aviris\NDVI\site3b_ndvi_output.tif


In [3]:
from osgeo import gdal
import numpy as np

# Open NDVI image
ndvi_path = r"D:\wenqu\chapter1_2\updated_aviris\NDVI\site3b_ndvi_output.tif"
ds = gdal.Open(ndvi_path)

# Read NDVI and NoData value
ndvi = ds.GetRasterBand(1).ReadAsArray().astype(float)
nodata = ds.GetRasterBand(1).GetNoDataValue()
if nodata is None:
    nodata = -9999

# Mask NoData first
ndvi_masked = np.where(ndvi == nodata, np.nan, ndvi)

# Create binary mask: 1 if NDVI > 0.1, else 0
binary_mask = np.where(ndvi_masked > 0.3, 1, 0).astype(np.uint8)

# Save binary mask
driver = gdal.GetDriverByName('GTiff')
out_path = r"D:\wenqu\chapter1_2\updated_aviris\NDVI\site3b_ndvi_mask.tif"
out_ds = driver.Create(out_path, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Byte)

# Copy spatial reference
out_ds.SetGeoTransform(ds.GetGeoTransform())
out_ds.SetProjection(ds.GetProjection())

# Write binary band
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(binary_mask)
out_band.SetNoDataValue(0)  # You can set NoData to 0 if you want to treat it as background

out_ds.FlushCache()
out_ds = None

print("Binary NDVI mask saved to:", out_path)


Binary NDVI mask saved to: D:\wenqu\chapter1_2\updated_aviris\NDVI\site3b_ndvi_mask.tif


In [69]:
from osgeo import gdal
import numpy as np

# Load image and mask
image_path  = r"D:\wenqu\chapter1_2\updated_aviris\AVIRIS_trait_map3\site6ab_d15n_std.tif"
mask_path  = r"D:\wenqu\chapter1_2\updated_aviris\NDVI\site6ab_ndvi_mask.tif"

# Open datasets
image_ds = gdal.Open(image_path)
mask_ds = gdal.Open(mask_path)

# Read bands
image = image_ds.GetRasterBand(1).ReadAsArray().astype(float)
mask = mask_ds.GetRasterBand(1).ReadAsArray().astype(float)

# Get NoData values
image_nodata = image_ds.GetRasterBand(1).GetNoDataValue()
mask_nodata = mask_ds.GetRasterBand(1).GetNoDataValue()

# Default NoData if not set
if image_nodata is None:
    image_nodata = -9999
if mask_nodata is None:
    mask_nodata = 0  # Binary mask often uses 0 as background

# Create a combined mask
combined_mask = (image == image_nodata) | (mask == mask_nodata)

# Multiply while preserving NoData
output = np.where(combined_mask, image_nodata, image * mask)

# Save output as GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_path = r'D:\wenqu\chapter1_2\updated_aviris\AVIRIS_trait_map3\mask\site6ab_d15n_std_masked.tif'
out_ds = driver.Create(out_path, image_ds.RasterXSize, image_ds.RasterYSize, 1, gdal.GDT_Float32)

# Copy spatial reference
out_ds.SetGeoTransform(image_ds.GetGeoTransform())
out_ds.SetProjection(image_ds.GetProjection())

# Write the result
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(output)
out_band.SetNoDataValue(image_nodata)

out_ds.FlushCache()
out_ds = None

print("Masked image saved to:", out_path)

Masked image saved to: D:\wenqu\chapter1_2\updated_aviris\AVIRIS_trait_map3\mask\site6ab_d15n_std_masked.tif
